In [1]:
import os
import json
from utils import *
import pandas as pd

In [15]:
# with open('kg.csv', 'r') as f:
#     lines = f.readlines()
#     headers = lines[0].strip().split(',')
#     for i, header in enumerate(headers):
#         header2idx[header] = i
#     for line_s in lines[1:]:
#         line = line_s.strip().split(',')
#         line = [l.strip() for l in line]
#         x = 'ENID'+str(line[header2idx['x_index']])
#         y = 'ENID'+str(line[header2idx['y_index']])
#         x_type = line[header2idx['x_type']]
#         y_type = line[header2idx['y_type']]
#         if y_type == 'DB00469':
#             print(line, len(line), line_s)
#         rel_name = line[header2idx['display_relation']]+'_'+y_type
#         actual_rels.add(line[header2idx['display_relation']].strip())
#         x_name = line[header2idx['x_name']]
#         y_name = line[header2idx['y_name']]
#         x_source = line[header2idx['x_source']]
#         y_source = line[header2idx['y_source']]
#         id2name[x] = x_name
#         id2name[y] = y_name
#         id2source[x] = x_source
#         id2source[y] = y_source
#         graph, rel_id_counter, rels, edge2source, id2name, id2source = insert_edge(x, y, rel_name, line, header2idx, graph, id2name, 
#                     rels, rel_id_counter, edge2src, line_s, id2source, x_name, y_name, x_source, y_source)
#         # if x_type != y_type:
#         #     rel_name = line[header2idx['display_relation']]+'_'+x_type
#         #     graph, rel_id_counter, rels, edge2source, id2name, id2source = insert_edge(y, x, rel_name, line, header2idx, graph, id2name, rels, 
#         #                 rel_id_counter, edge2src, line_s, id2source, y_name, x_name, y_source, x_source)

In [2]:
graph = {}
id2name = {}
id2source = {}
header2idx = {}
rel_id_counter = 0
rels = {}
edge2src = {}
actual_rels = set()

def insert_edge(x, y, rel_name, line, header2idx, graph, id2name, 
                rels, rel_id_counter, edge2source, line_s, id2source, x_name, y_name , x_source, y_source):
    if rel_name not in rels:
        rels[rel_name] = rel_id_counter
        rel_id = 'RIDR' + str(rel_id_counter)
        rel_id_counter += 1
    else:
        rel_id = 'RIDR' + str(rels[rel_name])
    
    if rel_id in id2name:
        if id2name[rel_id] != rel_name:
            print(rel_id, id2name[rel_id], rel_name)
        assert id2name[rel_id] == rel_name
    id2name[rel_id] = rel_name
    if x not in graph:
        graph[x] = {}
    if x not in edge2src:
        edge2src[x] = {}
    graph[x][y] = rel_id
    edge2src[x][y] = line_s
    
    return graph, rel_id_counter, rels, edge2source, id2name, id2source

df = pd.read_csv('kg.csv')

C:\Users\ved67\AppData\Local\Temp\ipykernel_18960\3926644955.py:33: DtypeWarning: Columns (3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('kg.csv')


In [3]:
for i, row in df.iterrows():
    x = 'ENID'+str(row['x_index'])
    y = 'ENID'+str(row['y_index'])
    x_type = row['x_type']
    y_type = row['y_type']
    rel_name = row['display_relation']+'_'+y_type
    actual_rels.add(row['display_relation'])
    x_name = row['x_name']
    y_name = row['y_name']
    x_source = row['x_source']
    y_source = row['y_source']
    id2name[x] = x_name
    id2name[y] = y_name
    id2source[x] = x_source
    id2source[y] = y_source
    graph, rel_id_counter, rels, edge2source, id2name, id2source = insert_edge(x, y, rel_name, row, df.columns, graph, id2name, 
                rels, rel_id_counter, edge2src, row, id2source, x_name, y_name, x_source, y_source)
    if x_type != y_type:
        rel_name = row['display_relation']+'_'+x_type
        graph, rel_id_counter, rels, edge2source, id2name, id2source = insert_edge(y, x, rel_name, row, df.columns, graph, id2name, rels, 
                    rel_id_counter, edge2src, row, id2source, y_name, x_name, y_source, x_source)

In [ ]:
actual_rels = list(actual_rels)
new_edgesrc = {}
for x in edge2src:
    new_edgesrc[x] = {}
    for y in edge2src[x]:
        new_edgesrc[x][y] = edge2src[x][y].to_dict()

In [ ]:
prime_folder = '/share/quacer_c/prime'
json.dump(graph, open(os.path.join(prime_folder, 'graph.json'), 'w'))
json.dump(id2name, open(os.path.join(prime_folder, 'id2name.json'), 'w'))
json.dump(id2source, open(os.path.join(prime_folder, 'id2source.json'), 'w'))
json.dump(rels, open(os.path.join(prime_folder, 'rels.json'), 'w'))
json.dump(edge2src, open(os.path.join(prime_folder, 'edge2src.json'), 'w'))
json.dump(actual_rels, open(os.path.join(prime_folder, 'actual_rels.json'), 'w'))
print('done')

In [2]:
prime_folder = '/share/quacer_c/prime'
actual_rels = json.load(open(os.path.join(prime_folder, 'actual_rels.json')))
edge2src = json.load(open(os.path.join(prime_folder, 'edge2src.json')))
graph = json.load(open(os.path.join(prime_folder, 'graph.json')))
id2name = json.load(open(os.path.join(prime_folder, 'id2name.json')))
id2source = json.load(open(os.path.join(prime_folder, 'id2source.json')))
rels = json.load(open(os.path.join(prime_folder, 'rels.json')))

In [3]:
entity_aliases = {}
relation_aliases = {}
for k, v in id2name.items():
    if k.startswith('RIDR'):
        relation_aliases[k] = [v]
    else:
        entity_aliases[k] = [v]

In [7]:
relation_aliases

{'RIDR0': ['ppi_gene/protein'],
 'RIDR1': ['carrier_gene/protein'],
 'RIDR2': ['enzyme_gene/protein'],
 'RIDR3': ['target_gene/protein'],
 'RIDR4': ['transporter_gene/protein'],
 'RIDR5': ['contraindication_disease'],
 'RIDR6': ['indication_disease'],
 'RIDR7': ['off-label use_disease'],
 'RIDR8': ['synergistic interaction_drug'],
 'RIDR9': ['associated with_effect/phenotype'],
 'RIDR10': ['parent-child_effect/phenotype'],
 'RIDR11': ['phenotype absent_effect/phenotype'],
 'RIDR12': ['phenotype present_effect/phenotype'],
 'RIDR13': ['phenotype present_disease'],
 'RIDR14': ['associated with_disease'],
 'RIDR15': ['parent-child_disease'],
 'RIDR16': ['side effect_effect/phenotype'],
 'RIDR17': ['parent-child_biological_process'],
 'RIDR18': ['parent-child_molecular_function'],
 'RIDR19': ['parent-child_cellular_component'],
 'RIDR20': ['interacts with_molecular_function'],
 'RIDR21': ['interacts with_cellular_component'],
 'RIDR22': ['interacts with_biological_process'],
 'RIDR23': ['i

In [8]:
def get_relation_template(relation_id):
    relation_templates = {
        # Gene/protein interactions
        'RIDR0': "{source} has protein-protein interaction with {target}",  # ppi_gene/protein
        'RIDR1': "{target} is a carrier of {source}",  # carrier_gene/protein
        'RIDR2': "{target} is an enzyme of {source}",  # enzyme_gene/protein
        'RIDR3': "{target} is a target of {source}",  # target_gene/protein
        'RIDR4': "{target} is a transporter of {source}",  # transporter_gene/protein
        
        # Drug-disease relationships
        'RIDR5': "{source} is contraindicated for {target}",  # contraindication_disease
        'RIDR6': "{source} is indicated for {target}",  # indication_disease
        'RIDR7': "{source} is used off-label for {target}",  # off-label use_disease
        
        # Drug interactions
        'RIDR8': "{source} has synergistic interaction with {target}",  # synergistic interaction_drug
        
        # Effect/phenotype relationships
        'RIDR9': "{target} is associated with {source}",  # associated with_effect/phenotype
        'RIDR10': "{target} includes {source}",  # parent-child_effect/phenotype
        'RIDR11': "{target} is absent in {source}",  # phenotype absent_effect/phenotype
        'RIDR12': "{target} is present in {source}",  # phenotype present_effect/phenotype
        'RIDR13': "{target} is present in {source}",  # phenotype present_disease
        'RIDR14': "{target} is associated with {source}",  # associated with_disease
        'RIDR15': "{target} includes {source}",  # parent-child_disease
        'RIDR16': "{target} is a side effect of {source}",  # side effect_effect/phenotype
        
        # Biological process relationships
        'RIDR17': "{target} includes {source}",  # parent-child_biological_process
        'RIDR18': "{target} includes {source}",  # parent-child_molecular_function
        'RIDR19': "{target} contains {source}",  # parent-child_cellular_component
        'RIDR20': "{target} interacts with {source}",  # interacts with_molecular_function
        'RIDR21': "{target} interacts with {source}",  # interacts with_cellular_component
        'RIDR22': "{target} interacts with {source}",  # interacts with_biological_process
        'RIDR23': "{target} interacts with {source}",  # interacts with_gene/protein
        
        # Disease and pathway relationships
        'RIDR24': "{target} is linked to {source}",  # linked to_disease
        'RIDR25': "{target} includes {source}",  # parent-child_exposure
        'RIDR26': "{target} includes {source}",  # parent-child_pathway
        'RIDR27': "{target} interacts with {source}",  # interacts with_pathway
        
        # Anatomical relationships
        'RIDR28': "{target} contains {source}",  # parent-child_anatomy
        'RIDR29': "{source} is expressed in {target}",  # expression present_anatomy
        'RIDR30': "{source} is not expressed in {target}",  # expression absent_anatomy
        
        # Drug mechanism relationships
        'RIDR31': "{target} is a carrier of {source}",  # carrier_drug
        'RIDR32': "{target} is an enzyme of {source}",  # enzyme_drug
        'RIDR33': "{target} is a target of {source}",  # target_drug
        'RIDR34': "{target} is a transporter of {source}",  # transporter_drug
        'RIDR35': "{target} is contraindicated for {source}",  # contraindication_drug
        'RIDR36': "{target} is indicated for {source}",  # indication_drug
        'RIDR37': "{target} is used off-label for {source}",  # off-label use_drug
        
        # Additional relationships
        'RIDR38': "{target} is associated with {source}",  # associated with_gene/protein
        'RIDR39': "{target} is absent in {source}",  # phenotype absent_disease
        'RIDR40': "{source} is a side effect of {target}",  # side effect_drug
        'RIDR41': "{target} interacts with {source}",  # interacts with_exposure
        'RIDR42': "{target} is linked to {source}",  # linked to_exposure
        'RIDR43': "{target} is expressed by {source}",  # expression present_gene/protein
        'RIDR44': "{target} is not expressed by {source}",  # expression absent_gene/protein
    }
    return relation_templates.get(relation_id, "{source} relates to {target}")

def create_graph_text_edge(graph, id2name):
    graph_text_edge = {}
    for source in graph:
        graph_text_edge[source] = {}
        for target in graph[source]:
            relation_id = graph[source][target]
            template = get_relation_template(relation_id)
            sentence = template.format(
                source=id2name[source],
                target=id2name[target]
            )
            graph_text_edge[source][target] = [sentence]
    
    return graph_text_edge
graph_text_edge = create_graph_text_edge(graph, id2name)
json.dump(graph_text_edge, open(os.path.join(prime_folder, 'graph_text_edge.json'), 'w'), indent=4)

In [4]:
# graph_text_edge = {}
# for x in graph:
#     graph_text_edge[x] = {}
#     for y in graph[x]:
#         sent = f'{id2name[x]} is related to {id2name[y]} by {id2name[graph[x][y]]}'
#         graph_text_edge[x][y] = [sent]

# list(list(graph_text_edge.values())[0].values())[:5]

[['PHYHIP is related to KIF15 by ppi_gene/protein'],
 ['PHYHIP is related to ZZEF1 by ppi_gene/protein'],
 ['PHYHIP is related to PNPLA2 by ppi_gene/protein'],
 ['PHYHIP is related to PAQR5 by ppi_gene/protein'],
 ['PHYHIP is related to CA10 by ppi_gene/protein']]

In [5]:
with open(os.path.join(prime_folder, 'graph_text_edge.json'), 'w') as f:
    json.dump(graph_text_edge, f)

In [ ]:
qa_algos = GraphAlgos(graph, entity_aliases, relation_aliases, allow_multiple_ans=True)
for i in range(5):
    query_results = qa_algos.generate_random_query(4, return_path=True)
    question_data, source, correct_answers, path_ids = query_results
    question = question_data[0]
    print(question)
    print([id2name[id] for id in path_ids], path_ids)

Path: ['ENID53774', 'ENID1207', 'ENID125013', 'ENID651', 'ENID68729']
Possible Paths: 134
ceramide binding->(interacts with_gene/protein) ->(interacts with_cellular_component) ->(interacts with_gene/protein) ->(expression present_anatomy) ->?
['ceramide binding', 'VDAC2', 'mitochondrion', 'PRKACA', 'adipose tissue of abdominal region'] ['ENID53774', 'ENID1207', 'ENID125013', 'ENID651', 'ENID68729']
Path: ['ENID35072', 'ENID29014', 'ENID3285']
Possible Paths: 124
LCT-AS1->(associated with_disease) ->(associated with_gene/protein) ->?
['LCT-AS1', 'Hirschsprung disease-ganglioneuroblastoma syndrome', 'CELSR3'] ['ENID35072', 'ENID29014', 'ENID3285']
Path: ['ENID97611', 'ENID35432', 'ENID97801', 'ENID35511', 'ENID30624']
Possible Paths: 170
HPA 1 Recognition polymorphism, beta-globin-related->(parent-child_disease) ->(parent-child_disease) ->(parent-child_disease) ->(parent-child_disease) ->?
['HPA 1 Recognition polymorphism, beta-globin-related', 'Mendelian disease', 'carnitine acetyltrans

In [22]:
for rel_id, rel_name in relation_aliases.items():
    if 'phenotype' in rel_name[0]:
        print(rel_id, rel_name)

RIDR9 ['associated with_effect/phenotype']
RIDR10 ['parent-child_effect/phenotype']
RIDR11 ['phenotype absent_effect/phenotype']
RIDR12 ['phenotype present_effect/phenotype']
RIDR13 ['phenotype present_disease']
RIDR16 ['side effect_effect/phenotype']
RIDR39 ['phenotype absent_disease']


In [20]:
context = ''
for node1, node_data in graph.items():
    all_drugs = []
    for node2, rel_id in node_data.items():
        if rel_id == 'RIDR36':
            all_drugs.append(node2)
    if len(all_drugs) > 1:
        num_side_effects = {}
        drug_context = ''
        for drug in all_drugs:
            side_effects = []
            for node2, rel_id in graph[drug].items():
                if rel_id == 'RIDR16':
                    side_effects.append(node2)
            num_side_effects[drug] = len(side_effects)
            drug_name = id2name[drug]
            drug_context += drug_name + ' has the following side effects: ' + ', '.join([id2name[side_effect] for side_effect in side_effects]) + '\n'
            if drug_name == 'Testosterone':
                print(len(side_effects))
            print(drug_name, len(side_effects))
        min_side_effects = min(num_side_effects.values())
        min_side_effects_drugs = [drug for drug, num in num_side_effects.items() if num == min_side_effects]
        if len(min_side_effects_drugs) == 1:
            drug_name = id2name[min_side_effects_drugs[0]]
            disease_name = id2name[node1]
            print(disease_name, drug_name, min_side_effects)
            print(drug_context)
            print(sum(num_side_effects.values()))
        break

126
Testosterone 126
Testosterone undecanoate 0
hypogonadotropic hypogonadism with or without anosmia Testosterone undecanoate 0
Testosterone has the following side effects: Abdominal pain, Aggressive behavior, Alopecia, Alopecia of scalp, Anaphylactic shock, Angina pectoris, Anxiety, Arthralgia, Arthritis, Back pain, Abnormality of coagulation, Abnormality of the coagulation cascade, Bronchitis, Chest pain, Confusion, Cough, Atopic dermatitis, Diplopia, Vertigo, Dyspnea, Respiratory distress, Dysuria, Eczema, Eczematoid dermatitis, Edema, Abnormality of the endocrine system, Inflammatory abnormality of the skin, Abnormality of the eye, Fatigue, Gastroesophageal reflux, Abnormality of the gastrointestinal tract, Gastrointestinal hemorrhage, Gingival recession, Gingivitis, Headache, Cardiac arrest, Congestive heart failure, Joint hemorrhage, Hematuria, Hirsutism, Hypercholesterolemia, Hyperesthesia, Hyperhidrosis, Hyperparathyroidism, Hyperventilation, Hypoglycemia, Hypogonadism, Neopla

In [21]:
disease_pairs = []
for node1, node_data in graph.items():
    diseases = []
    for node2, edge in node_data.items():
        if edge == 'RIDR13':
            diseases.append(node2)
    disease_pairs = []
    for i, disease1 in enumerate(diseases):
        for disease2 in diseases[i+1:]:
            disease_pairs.append((disease1, disease2))

SyntaxError: incomplete input (3054048617.py, line 1)

In [6]:
start_id = 27160
verts = {}
add = 0
while add < 100:
    start_id += add
    add +=1
    vert = 'ENID' + str(start_id)
    verts[vert] = len(graph[vert])
verts_l = sorted(list(verts.keys()), key=lambda x: x[1], reverse=True)
for vert in verts_l:
    print(vert, len(graph[vert]))
    break

ENID27160 12


In [7]:
en_id = 'ENID27160'
all_path_ids = []
print(len(graph[en_id]), en_id, id2name[en_id], id2source[en_id])
for i in range(5):
    query_results = qa_algos.generate_random_query(4, return_path=True,source=en_id)
    question_data, source, correct_answers, path_ids = query_results
    question = question_data[0]
    print(question) 
    print([id2name[id] for id in path_ids])
    all_path_ids.append(path_ids)

12 ENID27160 congenital stationary night blindness autosomal dominant MONDO_grouped
Path: ['ENID27160', 'ENID87156', 'ENID24394']
Possible Paths: 23
congenital stationary night blindness autosomal dominant->(phenotype present_effect/phenotype) ->(parent-child_effect/phenotype) ->?
['congenital stationary night blindness autosomal dominant', 'Bone spicule pigmentation of the retina', 'Pigmentary retinopathy']
Path: ['ENID27160', 'ENID35655', 'ENID29888', 'ENID2278']
Possible Paths: 150
congenital stationary night blindness autosomal dominant->(parent-child_disease) ->(parent-child_disease) ->(associated with_gene/protein) ->?
['congenital stationary night blindness autosomal dominant', 'autosomal dominant disease', 'centronuclear myopathy', 'TPM2']
Path: ['ENID27160', 'ENID22759']
Possible Paths: 6
congenital stationary night blindness autosomal dominant->(phenotype present_effect/phenotype) ->?
['congenital stationary night blindness autosomal dominant', 'Autosomal dominant inheritance

In [8]:
path_ids = all_path_ids[2]
for i in range(len(path_ids)-1):
    node1 = path_ids[i]
    node2 = path_ids[i+1]
    print(edge2src[node1][node2])

disease_phenotype_positive,phenotype present,27160,8099_12497_12498,disease,congenital stationary night blindness autosomal dominant,MONDO_grouped,22759,6,effect/phenotype,Autosomal dominant inheritance,HPO



In [9]:
actual_rels

{'associated with',
 'carrier',
 'contraindication',
 'enzyme',
 'expression absent',
 'expression present',
 'indication',
 'interacts with',
 'linked to',
 'off-label use',
 'parent-child',
 'phenotype absent',
 'phenotype present',
 'ppi',
 'side effect',
 'synergistic interaction',
 'target',
 'transporter'}

In [10]:
relation_aliases

{'RIDR0': ['ppi_gene/protein'],
 'RIDR1': ['carrier_gene/protein'],
 'RIDR2': ['carrier_111'],
 'RIDR3': ['carrier_7276'],
 'RIDR4': ['carrier_4\'-diol"'],
 'RIDR5': ['carrier_6\'-TETRAHYDROXY AURONE"'],
 'RIDR6': ['carrier_213'],
 'RIDR7': ['carrier_6906'],
 'RIDR8': ['carrier_DrugBank'],
 'RIDR9': ['carrier_6462'],
 'RIDR10': ['carrier_3-dien-1-yl]cyclohexene"'],
 'RIDR11': ['carrier_6'],
 'RIDR12': ['carrier_7450'],
 'RIDR13': ['carrier_2167'],
 'RIDR14': ['carrier_9'],
 'RIDR15': ['carrier_4-DIHYDRO-2H-CHROMEN-2-YL]METHOXY}BENZYL)-1'],
 'RIDR16': ['enzyme_gene/protein'],
 'RIDR17': ['enzyme_2147'],
 'RIDR18': ['enzyme_5743'],
 'RIDR19': ['enzyme_760'],
 'RIDR20': ['enzyme_5624'],
 'RIDR21': ['enzyme_126'],
 'RIDR22': ['enzyme_6817'],
 'RIDR23': ['enzyme_6783'],
 'RIDR24': ['enzyme_2645'],
 'RIDR25': ['enzyme_3098'],
 'RIDR26': ['enzyme_1363'],
 'RIDR27': ['enzyme_3416'],
 'RIDR28': ['enzyme_1588'],
 'RIDR29': ['enzyme_590'],
 'RIDR30': ['enzyme_1565'],
 'RIDR31': ['enzyme_6647'],
 

In [ ]:
nodes_question = {}
nodes_useful = set()

